In [14]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_replace, when, col

In [15]:
# Initialize SparkSession
spark = SparkSession.builder \
    .appName("DataCleaning") \
    .getOrCreate()

In [16]:

# Read the CSV file
df = spark.read.csv('./data/CO2.csv', header=True, inferSchema=True)

# Set the option to prevent escaping of special characters
spark.conf.set("spark.sql.sources.escapeQuotes", "false")
df.show()

+---+--------------------+-------------+---------------+-----------+
|_c0|     Marque / Modele|Bonus / Malus|Rejets CO2 g/km|Cout enerie|
+---+--------------------+-------------+---------------+-----------+
|  2|AUDI E-TRON SPORT...|    -6 000€ 1|              0|      319 €|
|  3|AUDI E-TRON SPORT...|    -6 000€ 1|              0|      356 €|
|  4|AUDI E-TRON 55 (4...|    -6 000€ 1|              0|      357 €|
|  5|AUDI E-TRON 50 (3...|    -6 000€ 1|              0|      356 €|
|  6|       BMW i3 120 Ah|    -6 000€ 1|              0|      204 €|
|  7|      BMW i3s 120 Ah|    -6 000€ 1|              0|      204 €|
|  8|    CITROEN BERLINGO|    -6 000€ 1|              0|      203 €|
|  9|      CITROEN C-ZERO|    -6 000€ 1|              0|      491 €|
| 10|DS DS3 CROSSBACK ...|    -6 000€ 1|              0|      251 €|
| 11|HYUNDAI KONA elec...|    -6 000€ 1|              0|      205 €|
| 12|HYUNDAI KONA elec...|    -6 000€ 1|              0|      205 €|
| 13|JAGUAR I-PACE EV4...|    -6 0

In [19]:


# Remove special characters from the 'Marque / Modele' column
df = df.withColumn("Marque / Modele", regexp_replace("Marque / Modele", "[^A-Za-z0-9\s]", ""))

# Remove the Euro sign (€) and any non-numeric characters from the 'Bonus / Malus' and 'Cout enerie' columns
# Remove everything after the Euro sign in the 'Bonus / Malus' column
df = df.withColumn("Bonus / Malus", regexp_replace("Bonus / Malus", "€.*1", ""))
#df = df.withColumn("Bonus / Malus", regexp_replace("Bonus / Malus", "-", "0"))
df = df.withColumn("Marque / Modele", regexp_replace("Marque / Modele", "[©Ã]", ""))
df = df.withColumn("Marque / Modele", regexp_replace("Marque / Modele", ",", ""))
df = df.withColumn("Bonus / Malus", regexp_replace("Bonus / Malus", "-", ""))
df = df.withColumn("Cout enerie", regexp_replace("Cout enerie", "[^\d]", ""))



# Remove unnecessary double quotes from 'Marque / Modele' column
df = df.withColumn("Marque / Modele", regexp_replace("Marque / Modele", '"', ''))
# Replace values containing only "-" with 0 in the 'Bonus / Malus' column
df = df.withColumn("Bonus / Malus", when(col("Bonus / Malus") == "-", 0).otherwise(col("Bonus / Malus")))

df.show()
# Write the cleaned data to a new CSV file
df.write.csv("Cleaned/cleaned_CO2_data.csv", header=True, mode="overwrite")


+---+--------------------+-------------+---------------+-----------+
|_c0|     Marque / Modele|Bonus / Malus|Rejets CO2 g/km|Cout enerie|
+---+--------------------+-------------+---------------+-----------+
|  2|AUDI ETRON SPORTB...|         6000|              0|        319|
|  3|AUDI ETRON SPORTB...|         6000|              0|        356|
|  4|AUDI ETRON 55 408...|         6000|              0|        357|
|  5|AUDI ETRON 50 313...|         6000|              0|        356|
|  6|       BMW i3 120 Ah|         6000|              0|        204|
|  7|      BMW i3s 120 Ah|         6000|              0|        204|
|  8|    CITROEN BERLINGO|         6000|              0|        203|
|  9|       CITROEN CZERO|         6000|              0|        491|
| 10|DS DS3 CROSSBACK ...|         6000|              0|        251|
| 11|HYUNDAI KONA elec...|         6000|              0|        205|
| 12|HYUNDAI KONA elec...|         6000|              0|        205|
| 13|JAGUAR IPACE EV40...|        

In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_replace, when

# Create a SparkSession
spark = SparkSession.builder \
    .appName("DataCleaning") \
    .getOrCreate()

# Read the CSV file into a Spark DataFrame
df = spark.read.csv("./data/CO2.csv", header=True)

# Remove the Euro sign (€) and any non-numeric characters from the 'Bonus / Malus' and 'Cout enerie' columns
df = df.withColumn("Marque / Modele", regexp_replace("Marque / Modele", "[©Ã]", ""))
df = df.withColumn("Marque / Modele", regexp_replace("Marque / Modele", ",", ""))
df = df.withColumn("Bonus / Malus", regexp_replace("Bonus / Malus", "[^\d-]", ""))
df = df.withColumn("Cout enerie", regexp_replace("Cout enerie", "[^\d]", ""))

# Replace empty values in 'Bonus / Malus' column with 0
df = df.withColumn("Bonus / Malus", when(df["Bonus / Malus"] == "-", 0).otherwise(df["Bonus / Malus"]))

# Remove unnecessary double quotes from 'Marque / Modele' column
df = df.withColumn("Marque / Modele", regexp_replace("Marque / Modele", '"', ''))

# Convert 'Bonus / Malus' and 'Cout enerie' columns to double data type
df = df.withColumn('Bonus / Malus', df['Bonus / Malus'].cast('double'))
df = df.withColumn('Cout enerie', df['Cout enerie'].cast('double'))

# Write the cleaned data to a new CSV file
df.write.csv("cleaned_data.csv", header=True, mode="overwrite")

df.show()


+---+--------------------+-------------+---------------+-----------+
|_c0|     Marque / Modele|Bonus / Malus|Rejets CO2 g/km|Cout enerie|
+---+--------------------+-------------+---------------+-----------+
|  2|AUDI E-TRON SPORT...|     -60001.0|              0|      319.0|
|  3|AUDI E-TRON SPORT...|     -60001.0|              0|      356.0|
|  4|AUDI E-TRON 55 (4...|     -60001.0|              0|      357.0|
|  5|AUDI E-TRON 50 (3...|     -60001.0|              0|      356.0|
|  6|       BMW i3 120 Ah|     -60001.0|              0|      204.0|
|  7|      BMW i3s 120 Ah|     -60001.0|              0|      204.0|
|  8|    CITROEN BERLINGO|     -60001.0|              0|      203.0|
|  9|      CITROEN C-ZERO|     -60001.0|              0|      491.0|
| 10|DS DS3 CROSSBACK ...|     -60001.0|              0|      251.0|
| 11|HYUNDAI KONA elec...|     -60001.0|              0|      205.0|
| 12|HYUNDAI KONA elec...|     -60001.0|              0|      205.0|
| 13|JAGUAR I-PACE EV4...|     -60

In [41]:

# Remove the euro sign (€) and spaces from the 'Bonus / Malus' and 'Cout enerie' columns
df = df.withColumn('Bonus / Malus', df['Bonus / Malus'].cast('string').cast('double'))
df = df.withColumn('Cout enerie', df['Cout enerie'].cast('string').cast('double'))


# Remove the Euro sign (€) and any non-numeric characters from the 'Bonus / Malus' and 'Cout enerie' columns
df = df.withColumn("Marque / Modele", regexp_replace("Marque / Modele", "[©Ã]", ""))
df = df.withColumn("Marque / Modele", regexp_replace("Marque / Modele", ",", ""))
df = df.withColumn("Bonus / Malus", regexp_replace("Bonus / Malus", "[^\d-]", ""))
df = df.withColumn("Cout enerie", regexp_replace("Cout enerie", "[^\d]", ""))

# Replace empty values in 'Bonus / Malus' column with 0
df = df.withColumn("Bonus / Malus", when(df["Bonus / Malus"] == "-", 0).otherwise(df["Bonus / Malus"]))

# Remove unnecessary double quotes from 'Marque / Modele' column
df = df.withColumn("Marque / Modele", regexp_replace("Marque / Modele", '"', ''))
df.show()
# Write the cleaned data to a new CSV file
df.write.csv("cleaned_data.csv", header=True, mode="overwrite")



+---+--------------------+-------------+---------------+-----------+
|_c0|     Marque / Modele|Bonus / Malus|Rejets CO2 g/km|Cout enerie|
+---+--------------------+-------------+---------------+-----------+
|  2|AUDI ETRON SPORTB...|     -6000100|              0|      31900|
|  3|AUDI ETRON SPORTB...|     -6000100|              0|      35600|
|  4|AUDI ETRON 55 408...|     -6000100|              0|      35700|
|  5|AUDI ETRON 50 313...|     -6000100|              0|      35600|
|  6|       BMW i3 120 Ah|     -6000100|              0|      20400|
|  7|      BMW i3s 120 Ah|     -6000100|              0|      20400|
|  8|    CITROEN BERLINGO|     -6000100|              0|      20300|
|  9|       CITROEN CZERO|     -6000100|              0|      49100|
| 10|DS DS3 CROSSBACK ...|     -6000100|              0|      25100|
| 11|HYUNDAI KONA elec...|     -6000100|              0|      20500|
| 12|HYUNDAI KONA elec...|     -6000100|              0|      20500|
| 13|JAGUAR IPACE EV40...|     -60

In [37]:
df.show()

+---+--------------------+-------------+---------------+-----------+
|_c0|     Marque / Modele|Bonus / Malus|Rejets CO2 g/km|Cout enerie|
+---+--------------------+-------------+---------------+-----------+
|  2|AUDI E-TRON SPORT...|      -600010|              0|       3190|
|  3|AUDI E-TRON SPORT...|      -600010|              0|       3560|
|  4|AUDI E-TRON 55 (4...|      -600010|              0|       3570|
|  5|AUDI E-TRON 50 (3...|      -600010|              0|       3560|
|  6|       BMW i3 120 Ah|      -600010|              0|       2040|
|  7|      BMW i3s 120 Ah|      -600010|              0|       2040|
|  8|    CITROEN BERLINGO|      -600010|              0|       2030|
|  9|      CITROEN C-ZERO|      -600010|              0|       4910|
| 10|DS DS3 CROSSBACK ...|      -600010|              0|       2510|
| 11|HYUNDAI KONA elec...|      -600010|              0|       2050|
| 12|HYUNDAI KONA elec...|      -600010|              0|       2050|
| 13|JAGUAR I-PACE EV4...|      -6

In [42]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Créer la session Spark
spark = SparkSession.builder \
    .appName("Count Null Rows") \
    .getOrCreate()



# Initialiser le compteur
total_null_rows = 0

# Parcourir toutes les colonnes du DataFrame
for col_name in df.columns:
    # Compter le nombre de lignes avec des valeurs nulles dans cette colonne
    null_rows_count = df.filter(df[col_name].isNull()).count()
    # Ajouter au compteur total
    total_null_rows += null_rows_count
    # Afficher le nombre de lignes avec des valeurs nulles dans cette colonne
    print("Nombre de lignes avec des valeurs nulles dans la colonne", col_name, ":", null_rows_count)

# Afficher le nombre total de lignes avec des valeurs nulles dans le DataFrame
print("Nombre total de lignes avec des valeurs nulles dans le DataFrame :", total_null_rows)



Nombre de lignes avec des valeurs nulles dans la colonne _c0 : 0
Nombre de lignes avec des valeurs nulles dans la colonne Marque / Modele : 0
Nombre de lignes avec des valeurs nulles dans la colonne Bonus / Malus : 0
Nombre de lignes avec des valeurs nulles dans la colonne Rejets CO2 g/km : 0
Nombre de lignes avec des valeurs nulles dans la colonne Cout enerie : 0
Nombre total de lignes avec des valeurs nulles dans le DataFrame : 0


In [32]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Créer la session Spark
spark = SparkSession.builder \
    .appName("Count Null Rows") \
    .getOrCreate()



# Initialiser le compteur
total_null_rows = 0

# Parcourir toutes les colonnes du DataFrame
for col_name in df.columns:
    # Compter le nombre de lignes avec des valeurs nulles dans cette colonne
    null_rows_count = df.filter(df[col_name].isNull()).count()
    # Ajouter au compteur total
    total_null_rows += null_rows_count
    # Afficher le nombre de lignes avec des valeurs nulles dans cette colonne
    print("Nombre de lignes avec des valeurs nulles dans la colonne", col_name, ":", null_rows_count)

# Afficher le nombre total de lignes avec des valeurs nulles dans le DataFrame
print("Nombre total de lignes avec des valeurs nulles dans le DataFrame :", total_null_rows)

# Write the cleaned data to a new CSV file
df.write.csv("Cleaned/CO2.csv", header=True, mode="overwrite")



Nombre de lignes avec des valeurs nulles dans la colonne _c0 : 0
Nombre de lignes avec des valeurs nulles dans la colonne Marque / Modele : 0
Nombre de lignes avec des valeurs nulles dans la colonne Bonus / Malus : 0
Nombre de lignes avec des valeurs nulles dans la colonne Rejets CO2 g/km : 0
Nombre de lignes avec des valeurs nulles dans la colonne Cout enerie : 0
Nombre total de lignes avec des valeurs nulles dans le DataFrame : 0
